In [1]:
import numpy as np
from src.Model import Model
from src.Convolution import Convolution
from src.Dense import Dense

In [2]:
model = Model()
model.add(Dense(num_units=5, activation_function="relu"))
model.add(Dense(num_units=3, activation_function="relu"))
model.add(Dense(num_units=1, activation_function="sigmoid"))

In [3]:
X = np.array([
    [1, 2, 3, 4, 5],
    [10, 20, 30, 40, 25],
    [3, 6, 7, 2, 4],
    [20, 10, 40, 25, 30],
    [4, 5, 6, 7, 8]
])
y = np.array([[0], [1], [0], [1], [0]])

In [4]:
model.fit(X, y, epochs=10, batch_size=3, learning_rate=0.1)

===== Epoch 1 =====
Accuracy: 0.6
===== Epoch 2 =====
Accuracy: 0.8
===== Epoch 3 =====
Accuracy: 0.8
===== Epoch 4 =====
Accuracy: 0.8
===== Epoch 5 =====
Accuracy: 0.8
===== Epoch 6 =====
Accuracy: 0.8
===== Epoch 7 =====
Accuracy: 0.8
===== Epoch 8 =====
Accuracy: 1.0
===== Epoch 9 =====
Accuracy: 1.0
===== Epoch 10 =====
Accuracy: 1.0


In [5]:
model.layers[0].weights

array([[ 0.26261453,  0.73789703, -1.19137339,  0.62416461, -0.88548471],
       [-0.71417534,  0.56528575, -0.93405883, -0.63906376,  0.68168348],
       [ 2.02069453,  0.52307537, -0.51567822, -2.50864859, -1.47684182],
       [-0.77206324, -0.0821996 ,  1.36003246,  0.45218796,  1.93646059],
       [ 0.32401295,  0.57877682, -0.77254716, -1.31368114, -0.96078189]])

In [6]:
model.predict([[4, 5, 6, 7, 8]])

array([[6.24162177e-05]])

## Convolution

In [7]:
import numpy as np
from src.Model import Model
from src.Convolution import Convolution
from src.Dense import Dense

In [8]:
Xm = np.array([
    [[[1,2,3],[4,5,6],[7,8,9]], [[10,11,12],[13,14,15],[16,17,18]], [[19,20,21], [22,23,24], [25,26,27]]]
])
ym = np.array([
    [[[1],[1]],
     [[1],[1]]]
])
input_shape = Xm[0].shape
m2 = Model()
m2.add(Convolution(input_size=input_shape, filter_size=(2, 2, 1), num_filters=1, padding_size=1, stride=2))

In [9]:
m2.fit(Xm, ym, epochs=10)

===== Epoch 1 =====
===== Epoch 2 =====
===== Epoch 3 =====
===== Epoch 4 =====
===== Epoch 5 =====
===== Epoch 6 =====
===== Epoch 7 =====
===== Epoch 8 =====
===== Epoch 9 =====
===== Epoch 10 =====


## Panda

In [15]:
# preprocess images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
batch_size = 25
img_height, img_width = 256, 256
input_shape = (img_height, img_width, 3)


def create_data_sparse():
    # data_gen = ImageDataGenerator(horizontal_flip=True, rotation_range=20, width_shift_range=0.2,
    #                               height_shift_range=0.2, zoom_range=0.2)
    data_gen = ImageDataGenerator(rescale = 1/255., validation_split = 0.2)

    train_images = data_gen.flow_from_directory('Dataset/PandasBears/Train', 
                                                target_size=(img_height, img_width),
                                                class_mode='binary', 
                                                batch_size=500, 
                                                subset='training', 
                                                # color_mode='rgb'
                                                )
    
    
    test_images = data_gen.flow_from_directory('Dataset/PandasBears/Test', 
                                               target_size=(img_height, img_width),
                                               class_mode='binary', 
                                               batch_size=200, 
                                               shuffle=False, 
                                            #    color_mode='rgb'
                                               )

    return train_images, test_images

train_images, test_images = create_data_sparse()

labels = ['bear', 'panda']

Found 400 images belonging to 2 classes.


Found 100 images belonging to 2 classes.


In [16]:
X_train = train_images[0][0][:8]
y_train = train_images[0][1][:8]

X_test = test_images[0][0]
y_test = test_images[0][1]

In [17]:
# import from src
from src.Model import Model
from src.Convolution import Convolution
from src.Dense import Dense
from src.Pooling import Pooling
from src.Flatten import Flatten

In [18]:
# Buat model, dengan weights bernilai random
samples = next(train_images)
input_shape = samples[0][0].shape

model = Model()
model.add(Convolution(input_size=input_shape, padding_size=1, filter_size=(3, 3, 3), num_filters=2, stride=1))
#model.add(Pooling(filter_size = (32,32, 2), stride=32, mode='max'))
model.add(Flatten())
model.add(Dense(num_units=7, activation_function="relu"))
model.add(Dense(num_units=1, activation_function="sigmoid"))

In [19]:
model.fit(X_train, y_train, epochs=2, batch_size=4)

d:\semester 7\ML\Tubes_CNN\src\Dense.py:97: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


===== Epoch 1 =====
Accuracy: 0.75
===== Epoch 2 =====
Accuracy: 1.0


In [20]:
from sklearn.model_selection import KFold
num_folds = 10
kfold = KFold(n_splits=num_folds, shuffle=True)
acc_per_fold = []
loss_per_fold = []



In [22]:
# Merge inputs and targets
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)
print(len(inputs))
print(len(targets))

108
108


In [23]:
# K-fold Cross Validation model evaluation
fold_no = 1

for train, test in kfold.split(inputs, targets):
     # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model
    history = model.fit(inputs[train], targets[train],
                epochs=2, batch_size=10)

------------------------------------------------------------------------
Training for fold 1 ...


d:\semester 7\ML\Tubes_CNN\src\Dense.py:97: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
